Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import sys

# # Add your folder to sys.path
# module_path = "/content/drive/MyDrive/Colab Notebooks/ML with Python Course/netflix"
# if module_path not in sys.path:
#     sys.path.append(module_path)

In [1]:
%pwd

'/content'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
file_path = f"{module_path}/netflix_incomplete.txt"
file_path =
df = pd.read_csv(file_path, sep=' ', header=None)
df = df.replace(0, np.nan)
df.columns = [f"c_{i}" for i in range(d)]

df_copy = df.copy()
n, d = df_copy.shape

In [ ]:
# Temporarily impute using mode
imputer = SimpleImputer(strategy='most_frequent')
X_temp = imputer.fit_transform(df)

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_temp)


In [ ]:
X_scaled.shape

In [ ]:
# ks = range(1, 15)
# bics = []
# aics = []

# for k in ks:
#     gmm = GaussianMixture(n_components=k, n_init=10, covariance_type='diag', random_state=42)
#     gmm.fit(X_scaled)
#     bics.append(gmm.bic(X_scaled))
#     aics.append(gmm.aic(X_scaled))

# # Plot to choose optimal K
# plt.plot(ks, bics, label='BIC')
# plt.plot(ks, aics, label='AIC')
# plt.xlabel('Number of Clusters (K)')
# plt.ylabel('Score')
# plt.title('Model Selection for GMM')
# plt.legend()
# plt.show()


In [ ]:
# fit GMM
k_chosen = 11
gmm = GaussianMixture(n_components=k_chosen, n_init=10, covariance_type="diag", random_state=42)
cluster_labels = gmm.fit_predict(X_scaled)

df_copy['cluster'] = cluster_labels

In [ ]:
# Cluster-based mean imputation
for feature in df_copy.columns[:-1]:
    for cluster_id in df_copy["cluster"].unique():
        cluster_mean = df_copy.loc[
            (df_copy[feature].notnull()) & (df_copy["cluster"] == cluster_id), feature
        ].mean()

        if pd.notna(cluster_mean):
            fill_value = int(round(cluster_mean))
        else:
            fallback_mean = df_copy[feature].mean()
            fill_value = int(round(fallback_mean))

        mask = (df_copy[feature].isnull()) & (df_copy["cluster"] == cluster_id)
        df_copy.loc[mask, feature] = int(round(fill_value))


In [ ]:
file_path = f"{module_path}/netflix_complete.txt"
df_true = pd.read_csv(file_path, sep=' ', header=None)


In [ ]:
df

In [ ]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import numpy as np

true_ratings, predicted_ratings = df_true.values, df_copy.iloc[:,:-1].values
rmse = root_mean_squared_error(true_ratings, predicted_ratings)
mae = mean_absolute_error(true_ratings, predicted_ratings)
r2 = r2_score(true_ratings, predicted_ratings)

print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# Mean Predictor
df_mean = df.fillna(df.mean())
true_ratings_baseline, predicted_ratings_baseline = \
    df_true.values, df_mean.values
rmse_b = root_mean_squared_error(true_ratings_baseline, predicted_ratings_baseline)
mae_b = mean_absolute_error(true_ratings_baseline, predicted_ratings_baseline)
r2_b = r2_score(true_ratings_baseline, predicted_ratings_baseline)

print(f"RMSE: {rmse_b:.4f}")
print(f"MAE : {mae_b:.4f}")
print(f"R² Score: {r2_b:.4f}")

In [ ]:
table = {
    "Model": ["RMSE", "MAE", "R² Score"],
    "Cluster_impt": [rmse, mae, r2],
    "Mean Baseline": [rmse_b, mae_b, r2_b]
}

pd.DataFrame(table)